In [ ]:
pip install moviepy

In [ ]:
pip install opencv-python

In [ ]:
pip install moviepy --upgrade

In [ ]:
import cv2
import glob
from moviepy.editor import *

In [ ]:
import tkinter as tk
import tkinter.filedialog
root = tk.Tk()
dir = 'C:'
file_path_in = tkinter.filedialog.askdirectory(initialdir = dir,title='参照するフォルダ')
file_path_out = tkinter.filedialog.askdirectory(initialdir = dir,title='保存するフォルダ')
root.withdraw()
print("参照するフォルダ:"+file_path_in)
print("保存するフォルダ:"+file_path_out)

In [ ]:
# opencvのmachiTemplateで画像比較
def match(img , temp):
#比較方法はcv2.TM_CCOEFF_NORMEDを選択
 result = cv2.matchTemplate(img, temp, cv2.TM_CCOEFF_NORMED)
#結果のmax_valが欲しい　0-1 1に近いほど似てる
 min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
 return (max_val)

In [ ]:
#動画切り出し
def triming(video, stime, etime):
    time_start = stime - 3
    if time_start < 0:
        time_start = 0
    time_end = etime - 2
    print(time_start,time_end)
    video = video.subclip(time_start, time_end)
    return video

In [ ]:
def movie_editor(file_path, k):
    video = cv2.VideoCapture(file_path)
    templateStart = cv2.imread("template.png", 0)
    frame_count = int(video.get(7)) #フレーム数を取得
    frame_rate = int(video.get(5)) #フレームレート(1フレームの時間単位はミリ秒)の取得 splatoon2は60
    killTime = 0 # 前回のチェックとの比較時間用。近いフレームは無視するようにする
    kill = [] # 取得タイミングをリストに入れる。秒単位
    stime = 0
    n=1 #n秒ごと

    for i in range(int((frame_count / frame_rate)/n)): #動画の秒数を取得し、回す
        video.set(1 ,frame_rate * n * i);
        _, frame = video.read() #動画をフレームに読み込み
        framegray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #比較用にグレー画像を作る
        checkVal = match(framegray,templateStart); #関数呼び出し
        if checkVal > 0.8: #キルログが出ている時間を取得
            killTime = i * n
            kill.append(killTime)
    print(kill)
    for time in kill:
        if stime == 0: #最初のキルログ
            stime = time
            etime = time
        elif (time-etime)>3:#キルログの間が3秒以上空いたとき
            if stime-2 < 0:
                stime = 2
            editor_video = VideoFileClip(file_path).subclip(stime-2,etime)
            save_path = file_path_out+"/kill_"+str(k)+".mp4"
            editor_video.write_videofile(save_path, fps=30)
            k += 1
            stime = 0
            etime = 0
        else: #キルログが出ている間
            etime = time
    if stime != 0:
        if stime - 2 < 0:
            stime = 2
        editor_video = VideoFileClip(file_path).subclip(stime-2,etime)
        save_path = file_path_out+"/kill_"+str(k)+".mp4"
        editor_video.write_videofile(save_path, fps=30)
        ktime = 0
        k += 1
    return k

In [ ]:
filepath_list = glob.glob(file_path_in+'/*.mp4')
k=1
for clip_path in filepath_list:
    k = movie_editor(clip_path, k)